### 1- Importing the needed libraries:

In [1]:
#needed installed libraries
# download first from here "https://chromedriver.storage.googleapis.com/index.html?path=94.0.4606.41/"
# then
# !pip install chromedriver_installer

In [3]:
# importing packages
import pandas as pd
import re

from bs4 import Tag, NavigableString, BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
import datetime
start_time = time()

from warnings import warn

/tmp/ipykernel_64651/660842286.py:7: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import clear_output


In [2]:
!pip3 install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 14.2 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 15.5 MB/s eta 0:00:00


### 2- Identifing the used lists:

In [4]:
titles=[]
jobs_lists=[]
names= []
locations=[]
timesdates=[]
datesandtimes=[]
job_desc=[]
level=[]
emp_type=[]
functions=[]
industries=[]
applicants=[]
salaries=[]

### 3- Scraping the pages with filters in LinkedIn.com:

In [5]:
# Insert the main link.
url = "https://www.linkedin.com/jobs/search?keywords=&location=Egypt&geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&sortBy=DD"
# this will open up new window with the url provided above 
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()
sleep(3)
action = ActionChains(driver)

# calculating the total jobs available in Egypt with the applied filters
noofjobs = int((driver.find_element_by_css_selector('h1>span').get_attribute('innerText')).replace("+","").replace(",",""))
print("the total jobs available in Egypt", noofjobs)
      
# to show more jobs. Depends on number of jobs selected
i = 2
no_of_jobs = 1050
while i <= int((no_of_jobs-25)/25)+1: 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    i = i + 1
    try:
        driver.find_element_by_xpath('//section[@class="two-pane-serp-page__results-list"]/button').click()
        sleep(3)

    except:
         pass
         sleep(3)


# parsing the visible webpage
pageSource = driver.page_source
lxml_soup = BeautifulSoup(pageSource, 'lxml')

# searching for all job containers
job_container = driver.find_element_by_class_name('jobs-search__results-list')
jobs = job_container.find_elements_by_tag_name('li') # return a list
print('We are scraping information about {} jobs.'.format(len(jobs)))
# searching for the jobs titles ,comanyies, locations and date and time
for job in jobs:
    job_title=job.find_element_by_css_selector("h3").get_attribute("innerText")
    titles.append(job_title)
    job_company=job.find_element_by_css_selector("h4").get_attribute("innerText")
    names.append(job_company)
    job_location=job.find_element_by_css_selector('div>div>span').get_attribute("innerText")
    locations.append(job_location)
    timedate=job.find_element_by_css_selector('div>div>time').get_attribute('innerText')
    timesdates.append(timedate)
    for i in timesdates:
        num = ""
        for c in i:
            if c.isdigit():
                 num = num + c    
    if (i.find('day') != -1):
            dt =  str(datetime.datetime.now() - datetime.timedelta(days=int(num)))
            datesandtimes.append(dt)
    elif (i.find('hour') != -1):
            dt =  str(datetime.datetime.now() - datetime.timedelta(hours=int(num)))
            datesandtimes.append(dt)
    elif (i.find('minute') != -1):
            dt =  str(datetime.datetime.now() - datetime.timedelta(minutes=int(num)))
            datesandtimes.append(dt)
    else:
            datesandtimes.append("Just now")
#start to navigate into each job and extract more details 

for item in range(len(jobs)):
     # clicking job to view job details
    job_click_path = f'/html/body/div/div/main/section/ul/li[{item+1}]'
    job_click = driver.find_element_by_xpath(job_click_path).click()
    sleep(2)
    # job description
    try:
        try:
            jobdesc_xpath = '/html/body/div/div/section/div[2]/section[2]/div[2]/section/div'
            job_descs = driver.find_element_by_xpath(jobdesc_xpath).text    
            job_desc.append(job_descs)
        except:
            jobdesc_xpath = '/html/body/div/div/section/div[2]/section[2]/div[2]/section/div/section/div/p[1]'
            job_descs = driver.find_element_by_xpath(jobdesc_xpath).text    
            job_desc.append(job_descs)
    except:
        job_desc.append("None")
    # Seniority level
    seniority_xpath = '/html/body/div/div/section/div[2]/section[2]/ul/li[1]/span'
    seniority = driver.find_element_by_xpath(seniority_xpath).text
    if seniority == "Full-time":
                level.append("None")
    elif seniority == "Part-time":
                level.append("None")
    else:
                level.append(seniority)

    # Employment type
    employment_type= driver.find_element_by_xpath('//h3[contains(text(),"Employment")]').text.strip()
    if employment_type  == "Employment type" :
        try:
                employment_type= driver.find_element_by_xpath('//span[contains(text(),"Full-time")]').text
                emp_type.append(employment_type)
        except:
                employment_type= driver.find_element_by_xpath('//span[contains(text(),"Part-time")]').text
                emp_type.append(employment_type)
    else:
            emp_type.append("None")
    
    # Job function
    try:
        function_xpath = '/html/body/div/div/section/div[2]/section[2]/ul/li[3]/span'
        job_function = driver.find_element_by_xpath(function_xpath).text
        functions.append(job_function)
    except:
        functions.append("None")
    # Industries
    try:
        industry_xpath = '/html/body/div/div/section/div[2]/section[2]/ul/li[4]/span'
        industry_type = driver.find_element_by_xpath(industry_xpath).text
        industries.append(industry_type)
    except:
        industries.append("None")
    # Applicants
    try:
        try:
            applicant = driver.find_element_by_xpath('//figcaption[contains(text(),"Be among the first 25 applicants")]')
            applicants.append("Less than 25")
        except:
            applicant_xpath = '/html/body/div/div/section/div[2]/section/div/div/div/h4/div/span'
            applicant = driver.find_element_by_xpath(applicant_xpath).text
            applicants.append(applicant)
    except:
        applicants.append("None")
    #salaries
    try:
        try:
            salary = driver.find_element_by_xpath('//span[contains(text(),"Salary")]')
            salaries.append(salary)
        except:
            salary = driver.find_element_by_xpath('//p[contains(text(),"Salary")]').text
            num=""
            for c in salary:
                if c.isdigit():
                          num = num + c  
            salaries.append(num)   
    except:
        salaries.append("Not avaliable")  
    item = item+1        
print("Titles list contains:",len(titles)) 
print("Seniority levels list contains:",len(level)) 
print("Job Titles Sample:",titles[:5])
print("Company names Sample:",names[:5])
print("Job location Sample:",locations[:5])
print("Date and Time Sample:",datesandtimes[:5])
print("Seniority levels Sample:",level[:5])
print("Employment type Sample:",emp_type[:5])
print("Sector and Industries Sample:",industries[:5])
print("Number of Applicants Sample:",applicants[:5])
print("Salaries Sample:",salaries[:5])

WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x564e7e156d93 <unknown>
#1 0x564e7df252d7 <unknown>
#2 0x564e7df4b51f <unknown>
#3 0x564e7df49a3d <unknown>
#4 0x564e7df8e4f4 <unknown>
#5 0x564e7df85353 <unknown>
#6 0x564e7df54e40 <unknown>
#7 0x564e7df56038 <unknown>
#8 0x564e7e1aa8be <unknown>
#9 0x564e7e1ae8f0 <unknown>
#10 0x564e7e18ef90 <unknown>
#11 0x564e7e1afb7d <unknown>
#12 0x564e7e180578 <unknown>
#13 0x564e7e1d4348 <unknown>
#14 0x564e7e1d44d6 <unknown>
#15 0x564e7e1ee341 <unknown>
#16 0x7fe1cb03eb43 <unknown>


In [5]:
# emp_type=[]
# # Insert the main link.
# url = "https://www.linkedin.com/jobs/search?keywords=&location=Egypt&geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&sortBy=DD"
# # this will open up new window with the url provided above 
# driver = webdriver.Chrome()
# driver.get(url)
# driver.maximize_window()
# sleep(3)
# action = ActionChains(driver)

# # calculating the total jobs available in Egypt with the applied filters
# noofjobs = int((driver.find_element_by_css_selector('h1>span').get_attribute('innerText')).replace("+","").replace(",",""))
# print("the total jobs available in Egypt", noofjobs)
      
# # to show more jobs. Depends on number of jobs selected
# i = 2
# no_of_jobs = 50
# while i <= int((no_of_jobs-25)/25)+1: 
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     i = i + 1
#     try:
#         driver.find_element_by_xpath('//section[@class="two-pane-serp-page__results-list"]/button').click()
#         sleep(3)

#     except:
#          pass
#          sleep(3)


# # parsing the visible webpage
# pageSource = driver.page_source
# lxml_soup = BeautifulSoup(pageSource, 'lxml')

# # searching for all job containers
# job_container = driver.find_element_by_class_name('jobs-search__results-list')
# jobs = job_container.find_elements_by_tag_name('li') # return a list
# for item in range(10):
#      # clicking job to view job details
#     job_click_path = f'/html/body/div/div/main/section/ul/li[{item+1}]'
#     job_click = driver.find_element_by_xpath(job_click_path).click()
#     sleep(3)
    
# #     employment_type= driver.find_element_by_xpath('//h3[contains(text(),"Employment")]').text.strip()
# #     if employment_type  == "Employment type" :
# #         employment_type= driver.find_element_by_xpath('//span[contains(text(),"Part")]').text
# #         emp_type.append(employment_type)
# #     else:
# # #         employment_type= driver.find_element_by_xpath('//span[contains(text(),"Full-time")]').text
# #         emp_type.append("Full-time")
#     employment_type= driver.find_element_by_xpath('//h3[contains(text(),"Employment")]').text.strip()
#     if employment_type  == "Employment type" :
#         try:
#                 employment_type= driver.find_element_by_xpath('//span[contains(text(),"Full-time")]').text
#                 emp_type.append(employment_type)
#         except:
#                 employment_type= driver.find_element_by_xpath('//span[contains(text(),"Part-time")]').text
#                 emp_type.append(employment_type)
#     else:
#             emp_type.append("None")
# print("Employment type Sample:",emp_type[:5])

In [6]:
# to check if we have all information
print(len(titles))
print(len(names))
print(len(locations))
print(len(datesandtimes))
print(len(timesdates))
print(len(salaries))
print(len(emp_type))
print(len(job_desc))
print(len(level))
print(len(industries))
print(len(applicants))
print(len(job_desc))

0
0
0
0
0
0
0
0
0
0
0
0


In [7]:
# creating a dataframe
job_data = pd.DataFrame({'Job Title': titles,
'Company Name': names,
'Location': locations,
'Date and Time': datesandtimes,
'Salary': salaries,
'Type': emp_type,
'Seniority Level': level,
'Industry': industries,
'Number of Applicants': applicants,
'Description': job_desc,
})

# cleaning description column
job_data['Description'] = job_data['Description'].str.replace('\n',' ')
print(job_data.info())
job_data.head()

AttributeError: Can only use .str accessor with string values!

### 5- Create csv file and fill it with values:

In [8]:
job_data.to_csv('F:\LinkedIn Job Data.csv', encoding='UTF-8-sig', index=0)